<a href="https://colab.research.google.com/github/asmasafdari/agentic-ai-langchain-project-chat-app/blob/main/Student_Helper_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install -q langchain langchain-google-genai langchain_community sentence-transformers langchain-chroma

In [69]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Load your Google Generative AI API key
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')  # Securely load API key
# GEMINI_API_KEY = "your_api_key_here"  # Alternatively, hard-code your API key

# Initialize the Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Gemini model
    key=GEMINI_API_KEY,
    temperature=0  # Deterministic responses
)

In [70]:
response = llm.predict("Hello! Can you help me create a study plan?")
print(response)


<ipython-input-70-468e782364ad>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict("Hello! Can you help me create a study plan?")
ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d22ef3e0220>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/g

KeyboardInterrupt: 

In [61]:
# from langchain.memory import ConversationBufferMemory

# short_term_memory = ConversationBufferMemory()


In [62]:
# from langchain_chroma import Chroma
# from langchain.memory import ConversationSummaryBufferMemory
# from sentence_transformers import SentenceTransformer


# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# vector_store = Chroma(embedding_function=embedding_model)

# long_term_memory = ConversationSummaryBufferMemory(
#     llm=llm,
#     retriever=vector_store.as_retriever(),
#     memory_key="long_term_memory"
# )


In [65]:
# from langchain.chains import ConversationChain

# class DualMemoryChatbot:
#     def __init__(self, llm, short_term_memory, long_term_memory):
#         self.conversation_chain = ConversationChain(
#             llm=llm,
#             memory=short_term_memory
#         )
#         self.long_term_memory = long_term_memory

#     def chat(self, user_input):
#         response = self.conversation_chain.predict(input=user_input)

#         self.long_term_memory.save_context(
#             {"input": user_input},
#             {"output": response}
#         )

#         return response

#     def recall_long_term_memory(self, query):
#         return self.long_term_memory.load_memory_variables({"input": query})


# chatbot = DualMemoryChatbot(
#     llm=llm,
#     short_term_memory=short_term_memory,
#     long_term_memory=long_term_memory
# )

from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain.memory import ConversationSummaryBufferMemory
from sentence_transformers import SentenceTransformer
from langchain.chains import ConversationChain

# Initialize short-term memory
short_term_memory = ConversationBufferMemory()

# Initialize embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize Chroma vector store
vector_store = Chroma(embedding_function=embedding_model)

# Initialize long-term memory
long_term_memory = ConversationSummaryBufferMemory(
    llm=llm,
    retriever=vector_store.as_retriever(),
    memory_key="long_term_memory"
)

# Define the chatbot class with dual memory
class DualMemoryChatbot:
    def __init__(self, llm, short_term_memory, long_term_memory):
        self.conversation_chain = ConversationChain(
            llm=llm,
            memory=short_term_memory
        )
        self.long_term_memory = long_term_memory

    def chat(self, user_input):
        # Generate response using short-term memory
        response = self.conversation_chain.predict(input=user_input)

        # Save conversation to long-term memory
        self.long_term_memory.save_context(
            {"input": user_input},
            {"output": response}
        )
        return response

    def recall_long_term_memory(self, query):
        # Retrieve relevant information from long-term memory
        return self.long_term_memory.load_memory_variables({"input": query})


# Initialize the chatbot
chatbot = DualMemoryChatbot(
    llm=llm,
    short_term_memory=short_term_memory,
    long_term_memory=long_term_memory
)


In [66]:
def generate_prompt(user_input, conversation_history, long_term_context):
    base_prompt = f"""
    You are a helpful AI assistant designed for students specifically. You assist at you best capabilities with:
    1. Researching topics for exams and explaining them in the simplest way with the best keywords to score well.
    2. Creating detailed study schedules for proper time management.

    Short-term conversation history: {conversation_history}
    Relevant long-term memory: {long_term_context}

    Student says: {user_input}
    """
    return base_prompt


In [67]:
def chat(self, user_input):
    long_term_context = self.recall_long_term_memory(user_input)

    conversation_history = self.conversation_chain.memory.chat_memory.messages
    prompt = generate_prompt(user_input, conversation_history, long_term_context)

    response = self.conversation_chain.llm.predict(prompt)

    self.long_term_memory.save_context(
        {"input": user_input},
        {"output": response}
    )

    return response


In [47]:
def generate_study_plan(subjects, hours_per_day):
    study_plan = []
    for subject in subjects:
        study_plan.append(f"Spend {hours_per_day / len(subjects):.1f} hours on {subject} each day.")
    return "\n".join(study_plan)

subjects = ["Physics", "Chemistry", "Math"]
hours = 6
print(generate_study_plan(subjects, hours))


Spend 2.0 hours on Physics each day.
Spend 2.0 hours on Chemistry each day.
Spend 2.0 hours on Math each day.


In [ ]:
# Testing
print(chatbot.chat("Hi, can you help me create a study plan for Physics?"))
print(chatbot.chat("I have exams in 2 weeks."))


print(chatbot.recall_long_term_memory("What study plan did I create for Physics?"))
